In [43]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [44]:
df = pd.read_csv('data.csv')

C:\Users\valer\AppData\Local\Temp\ipykernel_18008\1507164070.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [45]:
df.info( show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   maker                3033997 non-null  object 
 1   model                2419551 non-null  object 
 2   mileage              3190328 non-null  float64
 3   manufacture_year     3182334 non-null  float64
 4   engine_displacement  2809498 non-null  float64
 5   engine_power         2998035 non-null  float64
 6   body_type            2429998 non-null  object 
 7   color_slug           209501 non-null   object 
 8   stk_year             536105 non-null   float64
 9   transmission         2811282 non-null  object 
 10  door_count           2462846 non-null  float64
 11  seat_count           2265813 non-null  float64
 12  fuel_type            1705306 non-null  object 
 13  date_created         3552912 non-null  object 
 14  date_last_seen       3552912 non-null  object 
 15

In [46]:
df = df[(df['price_eur'] >= 500) & (df['price_eur'] <= 30_000_000)]
df = df.reset_index(drop=True)

In [47]:
df = df.drop('stk_year', axis = 1)
df = df.drop('color_slug', axis = 1)

In [48]:
null_percentage = df.isnull().mean() * 100
null_percentage

maker                  14.100859
model                  31.455159
mileage                 8.796693
manufacture_year        9.468416
engine_displacement    20.143503
engine_power           14.394111
body_type              32.083728
transmission           19.600642
door_count             29.527101
seat_count             35.169624
fuel_type              52.834753
date_created            0.000000
date_last_seen          0.000000
price_eur               0.000000
dtype: float64

In [ ]:
categorical_columns = ['maker', 'model', 'body_type', 'transmission', 'fuel_type']

for col in categorical_columns:
    mean_price_by_category = df.groupby(col)['price_eur'].mean()
    df[col] = df[col].map(mean_price_by_category)

In [ ]:
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_last_seen'] = pd.to_datetime(df['date_last_seen'], errors='coerce')


df['days_diff'] = (df['date_last_seen'] - df['date_created']).dt.days


df.drop(columns=['date_created', 'date_last_seen'], inplace=True)

In [51]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=42)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

c:\Users\valer\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [52]:
from imblearn.over_sampling import SMOTE

In [53]:
X = df_imputed.drop(columns=['price_eur'])
y = df_imputed['price_eur']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [54]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import smogn

In [ ]:

train_df = X_train.copy()
train_df['price_eur'] = y_train
train_df = train_df.reset_index(drop=True)

train_df_smogn = smogn.smoter(
    data=train_df,
    y='price_eur',
    rel_thres=0.8,        
    rel_method='auto',   
    rel_coef=0.5         
)


MemoryError: Unable to allocate 2.11 TiB for an array with shape (538871, 538871) and data type float64

In [ ]:

X_train_smogn = train_df_smogn.drop(columns=['price_eur'])
y_train_smogn = train_df_smogn['price_eur']



In [ ]:

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_smogn)
X_test_scaled = scaler.transform(X_test)

In [ ]:

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_scaled, y_train_smogn)


y_pred = rf.predict(X_test_scaled)


In [ ]:


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
threshold = 15
accuracy = np.mean(np.abs(y_pred - y_test) <= threshold) * 100  # in percentage

print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Threshold Accuracy (within {threshold} units): {accuracy:.2f}%")

R² Score: 0.3696
Mean Absolute Error (MAE): 7538.65
Mean Squared Error (MSE): 3601475583.69
Root Mean Squared Error (RMSE): 60012.30
Threshold Accuracy (within 15 units): 0.51%
